In [23]:
# Library import

import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import pandas as pd
from pathlib import Path
import datetime as dt
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from google.cloud import bigquery
from google.oauth2 import service_account

delta=dt.timedelta(days=1)

In [24]:
# Get main path and other folders
s_mainpath=Path.cwd()
print(s_mainpath)
s_parentpath=s_mainpath.parent.parent.parent
print(s_parentpath)
# folderpath=s_parentpath.joinpath(r'DatosXM\Despacho\\')

# Ruta General
sRutaDesp=r'C:\Información XM\\Publico\\'

sRutaPrint=s_mainpath.joinpath('Print')

c:\Alejo\Eje de Planeación\Análisis Eléctrico\ScriptsPython
c:\Alejo


In [25]:
# Rango de análisis

s_FechaIni='2025-08-22'
s_FechaFin='2025-08-22'

#Cambiar a formato fecha
d_FechaIni=dt.datetime.strptime(s_FechaIni,'%Y-%m-%d')
d_FechaFin=dt.datetime.strptime(s_FechaFin,'%Y-%m-%d')

In [26]:
# Inicializar fecha
d_Fecha=d_FechaIni

# Encabezados 
l_colGen=['Planta'] 
l_colCf=['Planta'] 

for i in range(1,25): 
    l_colGen.append(i)
    l_colCf.append(i)

# filetypes=['dSEGDES','dDEM','dINT','dLIM','dDEC'] # Lista con los archivos a analizar
# filetypes=['dDEC','dConFig','rDEC','rConFig'] # Lista con los archivos a analizar
filetypes=['dDEC','dConFig'] # Lista con los archivos a analizar

# l_SubCar2=['ATLANTIC','BOLIVAR','GCM'] # Lista con las subareas

# l_SubCar=['ATLANTIC','BOLIVAR','GCM','CORDOSUC','CERROMAT'] # Lista con las subareas

df_Conf=pd.DataFrame()
df_Gen=pd.DataFrame()
df_ConfR=pd.DataFrame()
df_GenR=pd.DataFrame()


while d_Fecha<=d_FechaFin:

    ano=d_Fecha.year
    mes=d_Fecha.month
    dia=d_Fecha.day

    for filetype in filetypes:

        if filetype=='dDEC':
            folderpath=Path(sRutaDesp + 'Despacho\\')
            file=filetype + "{:02d}".format(mes) + "{:02d}".format(dia) + '.txt'
            s_filepath=folderpath.joinpath(str(ano) + '-' + "{:02d}".format(mes),file)
            df_Aux = pd.read_csv(s_filepath, sep=',',names=l_colGen,encoding="ISO-8859-1")
            df_Aux['Fecha']=d_Fecha
            df_Aux2=df_Aux[df_Aux['Planta'].isin(['TEBSAB CC','TERMOCANDELARIA CC','BARRANQUILLA 3','BARRANQUILLA 4'])]
            # df_Aux2=df_Aux[df_Aux['Planta'].isin(['TEBSA','TCANDELCC','BARRANQ3','BARRANQ4'])]
            # df_Aux2=df_Aux[df_Aux['Nombre'].isin(['TEBSA','FLORES IV','FLORES1'])]
            df_Gen=pd.concat([df_Gen,df_Aux2])

        elif filetype=='dConFig':
            folderpath=Path(sRutaDesp + 'Despacho\\')
            file=filetype + "{:02d}".format(mes) + "{:02d}".format(dia) + '.txt'
            s_filepath=folderpath.joinpath(str(ano) + '-' + "{:02d}".format(mes),file)
            df_Aux = pd.read_csv(s_filepath, sep=',',names=l_colCf,encoding="ISO-8859-1")
            df_Aux['Fecha']=d_Fecha
            df_Conf=pd.concat([df_Conf,df_Aux])

        elif filetype=='rDEC':
            folderpath=Path(sRutaDesp + 'Redespacho\\')
            file=filetype + "{:02d}".format(mes) + "{:02d}".format(dia) + '.txt'
            s_filepath=folderpath.joinpath(str(ano) + '-' + "{:02d}".format(mes),file)
            df_Aux = pd.read_csv(s_filepath, sep=',',names=l_colGen,encoding="ISO-8859-1")
            df_Aux['Fecha']=d_Fecha
            df_Aux2=df_Aux[df_Aux['Planta'].isin(['TEBSAB CC','TERMOCANDELARIA CC','BARRANQUILLA 3','BARRANQUILLA 4'])]
            # df_Aux2=df_Aux[df_Aux['Nombre'].isin(['TEBSA','FLORES IV','FLORES1'])]
            df_GenR=pd.concat([df_GenR,df_Aux2])

        elif filetype=='rConFig':
            folderpath=Path(sRutaDesp + 'Redespacho\\')
            file=filetype + "{:02d}".format(mes) + "{:02d}".format(dia) + '.txt'
            s_filepath=folderpath.joinpath(str(ano) + '-' + "{:02d}".format(mes),file)
            df_Aux = pd.read_csv(s_filepath, sep=',',names=l_colCf,encoding="ISO-8859-1")
            df_Aux['Fecha']=d_Fecha
            df_ConfR=pd.concat([df_ConfR,df_Aux])

        # elif filetype=='dDEM':
        #     folderpath=Path(sRutaDesp + 'Despacho\\')
        #     file=filetype + "{:02d}".format(mes) + "{:02d}".format(dia) + '.txt'
        #     s_filepath=folderpath.joinpath(str(ano) + '-' + "{:02d}".format(mes),file)
        #     df_Aux = pd.read_csv(s_filepath, sep=',',names=l_colDem,encoding="ISO-8859-1")
        #     df_Aux['Fecha']=d_Fecha
        #     df_Aux2=df_Aux[df_Aux['NombreSub'].isin(l_SubCar2)]
        #     df_DemSubCar2=pd.concat([df_DemSubCar2,df_Aux2])
        #     df_Aux2=df_Aux[df_Aux['NombreSub'].isin(l_SubCar)]
        #     df_DemSubCar=pd.concat([df_DemSubCar,df_Aux2])
        #     df_Dem=pd.concat([df_Dem,df_Aux])

        # elif filetype=='dINT':
        #     folderpath=Path(sRutaDesp + 'Despacho\\')
        #     file=filetype + "{:02d}".format(mes) + "{:02d}".format(dia) + '.txt'
        #     s_filepath=folderpath.joinpath(str(ano) + '-' + "{:02d}".format(mes),file)
        #     df_Aux = pd.read_csv(s_filepath, sep=',',names=l_colDem,encoding="ISO-8859-1")
        #     df_Aux['Fecha']=d_Fecha
        #     df_Aux2=df_Aux[df_Aux['NombreSub'].isin(['CARIBE2'])]
        #     df_Int=pd.concat([df_Int,df_Aux2])

        # elif filetype=='dLIM':
        #     folderpath=Path(sRutaDesp + 'Despacho\\')
        #     file=filetype + "{:02d}".format(mes) + "{:02d}".format(dia) + '.txt'
        #     s_filepath=folderpath.joinpath(str(ano) + '-' + "{:02d}".format(mes),file)
        #     df_Aux = pd.read_csv(s_filepath, sep=',',names=l_colDem,encoding="ISO-8859-1")
        #     df_Aux['Fecha']=d_Fecha
        #     df_Aux2=df_Aux[df_Aux['NombreSub'].isin(['CARIBE2'])]
        #     df_Limp=pd.concat([df_Limp,df_Aux2])



    d_Fecha=d_Fecha+delta

In [27]:
# Función para asignar los días de la semana a cada fecha, si es festivo se trata como un domingo
import holidays
co_holidays = holidays.Colombia()

def typedays(row,weekday):

     if weekday==True:
          return row['Fecha'].weekday()
     else:
          if row['Fecha'] in co_holidays:
               return 'F'
          elif row['Fecha'].weekday()==5:
               return 'S'
          elif row['Fecha'].weekday()==6:
               return 'F'
          else:
               return 'O'

In [28]:
df_GenP=df_Gen.copy()
df_GenP=pd.melt(df_GenP, id_vars=['Fecha','Planta'], var_name='Periodo', value_name='MW')

df_ConfP=df_Conf.copy()
df_ConfP=pd.melt(df_ConfP, id_vars=['Fecha','Planta'], var_name='Periodo', value_name='Conf')

df_GenP=df_GenP.merge(df_ConfP,left_on=['Fecha','Planta','Periodo'],right_on=['Fecha','Planta','Periodo'], how='left')[['Fecha','Planta','Periodo','MW','Conf']]

# Definir tipo de día
df_GenP['day_osf']=df_GenP.apply(lambda row: typedays(row,weekday=False),axis=1)

df_GenP=df_GenP[(df_GenP['Planta'].isin(['TERMOCANDELARIA CC','TEBSAB CC']))]
# df_GenP=df_GenP[(df_GenP['Planta'].isin(['TCANDELCC','TEBSA']))]
df_GenP=df_GenP.sort_values(by=['Fecha','Planta','Periodo'],ascending=[True,True,True])

df_GenP.to_csv('GenTPL.csv')

df_GenP

,Fecha,Planta,Periodo,MW,Conf,day_osf
2,2025-08-22,TEBSAB CC,1,437.00,7,O
6,2025-08-22,TEBSAB CC,2,234.00,4,O
10,2025-08-22,TEBSAB CC,3,210.00,2,O
14,2025-08-22,TEBSAB CC,4,210.00,2,O
18,2025-08-22,TEBSAB CC,5,225.72,2,O
22,2025-08-22,TEBSAB CC,6,210.00,2,O
26,2025-08-22,TEBSAB CC,7,210.00,2,O
30,2025-08-22,TEBSAB CC,8,210.00,2,O
34,2025-08-22,TEBSAB CC,9,210.00,2,O
38,2025-08-22,TEBSAB CC,10,210.00,2,O


In [29]:
df_GenPR=df_GenR.copy()
df_GenPR=pd.melt(df_GenPR, id_vars=['Fecha','Planta'], var_name='Periodo', value_name='MW')


df_ConfPR=df_ConfR.copy()
df_ConfPR=pd.melt(df_ConfPR, id_vars=['Fecha','Planta'], var_name='Periodo', value_name='Conf')

df_GenPR=df_GenPR.merge(df_ConfPR,left_on=['Fecha','Planta','Periodo'],right_on=['Fecha','Planta','Periodo'], how='left')[['Fecha','Planta','Periodo','MW','Conf']]

# Definir tipo de día
df_GenPR['day_osf']=df_GenPR.apply(lambda row: typedays(row,weekday=False),axis=1)

df_GenPR=df_GenPR[(df_GenPR['Planta'].isin(['TCANDELCC','TEBSA']))]
df_GenPR=df_GenPR.sort_values(by=['Fecha','Planta','Periodo'],ascending=[True,True,True])

df_GenPR.to_csv('GenTPLRed.csv')

df_GenPR

KeyError: "The following 'id_vars' are not present in the DataFrame: ['Fecha', 'Planta']"